In [28]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import time

In [2]:
def get_data(url):
    hdr = {'User-Agent': 'Mozilla/5.0'}
    html = requests.get(url, headers=hdr).content
    soup=BeautifulSoup(html)
    name=soup.find('span', {'itemprop':'name'}).text
    df=pd.DataFrame({'name':name}, index=[0])
    for i in range(7,10):
        df_aux=pd.read_html(html)[7].T
        df_aux.columns=df_aux.iloc[0,:]
        df_aux=df_aux.loc[1,:]
        df_aux=pd.DataFrame(df_aux).T
        df_aux=df_aux.reset_index(drop=True)
        df=pd.concat([df,df_aux], axis=1)
    return df

In [3]:
 def get_players_link():
    url = 'https://fmdataba.com/20/l/191/bundesliga/best-players/'
    hdr = {'User-Agent': 'Mozilla/5.0'}
    html = requests.get(url, headers=hdr).content
    soup=BeautifulSoup(html)
    teams=soup.find_all('table', {'class':'table table-striped'})[1].find_all('td')
    url_teams=[item.find('a')['href'] for item in teams]
    url_list = []
    for team in url_teams:
        html = requests.get(team, headers=hdr).content
        soup=BeautifulSoup(html)
        players=soup.find('table', {'id':'tablo61'}).find_all('td', {'style':'font-size:12px;'})
        url_players=[item.find('a')['href'] for item in players]
        url_list.append(url_players)
    url_list=[item for x in url_list for item in x]
    return url_list

In [9]:
urls = get_players_link()

In [12]:
df = pd.DataFrame({'url':urls})

In [15]:
df_sample = df.sample(20)

In [29]:
df_result = pd.DataFrame()
for url in tqdm(df_sample.url):
    time.sleep(10)
    try:
        df_aux = get_data(url)
        df_result = pd.concat([df_result, df_aux])
    except:
        pass

 25%|██▌       | 5/20 [00:58<02:56, 11.76s/it]C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  
100%|██████████| 20/20 [05:50<00:00, 17.54s/it]


In [26]:
df_result

,name,Corners,Crossing,Dribbling,Finishing,First Touch,Free Kick,Heading,Long Shots,Long Throws,...,First Touch,Free Kick,Heading,Long Shots,Long Throws,Marking,Passing,Penalty Taking,Tackling,Technical
0,Nico Schulz,10,15,12,9,11,9,8,9,14,...,11,9,8,9,14,10,12,8,13,11
0,Philip Fontein,5,7,9,4,11,7,7,10,4,...,11,7,7,10,4,8,9,2,11,6
0,John Iredale,8,6,14,10,14,8,14,9,4,...,14,8,14,9,4,5,9,3,5,14
0,Marcel Sabitzer,9,11,13,15,12,13,8,11,6,...,12,13,8,11,6,8,12,10,10,13
0,Sheraldo Becker,6,10,13,9,13,10,8,8,5,...,13,10,8,8,5,4,10,9,5,12
0,André Hoffmann,6,7,7,7,10,7,12,8,12,...,10,7,12,8,12,14,11,9,15,10
0,Gabriel Boakye,7,11,14,13,10,4,11,6,2,...,10,4,11,6,2,6,8,4,6,11
0,Tim Hüttl,5,1,6,4,10,2,11,6,8,...,10,2,11,6,8,15,9,4,15,7
0,Ayman Azhil,8,8,9,6,11,7,7,7,4,...,11,7,7,7,4,9,12,7,12,11
0,Max Klump,11,13,12,3,13,8,1,6,1,...,13,8,1,6,1,4,8,5,7,12


In [24]:
df_sample

,url
226,https://fmdataba.com/20/p/147629/nico-schulz/
690,https://fmdataba.com/20/p/177305/philip-fontein/
804,https://fmdataba.com/20/p/186535/john-iredale/
521,https://fmdataba.com/20/p/147392/marcel-sabitzer/
764,https://fmdataba.com/20/p/151837/sheraldo-becker/
374,https://fmdataba.com/20/p/150110/tomas-koubek/
856,https://fmdataba.com/20/p/153175/stefanos-kapino/
430,https://fmdataba.com/20/p/150362/andre-hoffmann/
48,https://fmdataba.com/20/p/184940/gabriel-boakye/
734,https://fmdataba.com/20/p/185984/tim-huttl/
